## Extract features for sepsis patients received mechanical ventilation. Some tables are obtained by running the official GitHub repo.

In [1]:
#libraries
import numpy as np
import pandas as pd
import psycopg2 #used to connect to our local MIMIC-III database
import collections
# import getpass
from datetime import datetime
import os,sys,re
import pickle
import csv
import math
#import seaborn as sns
# import random
from datetime import timedelta
from pathlib import Path
import importlib
import bisect
import glob
import copy


from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

#%load_ext autotime

{'width': 1024, 'height': 768, 'scroll': True}

In [2]:
dbname = 'mimiciii'
password = '15289943821'
user = 'postgres'
conn = psycopg2.connect(dbname=dbname, password=password,user=user,port=5433)
cur=conn.cursor()

### Select sepsis cohort

In [3]:
query_sepsis = """
select ss.icustay_id, ss.hadm_id, ss.age, ss.is_male, ss.race_white, ss.race_black, ss.race_hispanic,
ss.race_other,  ss.height, mv.weight
from public.sepsis3 ss join (select DISTINCT mv.icustay_id, mv.weight as weight from public.weightfirstday mv) mv on ss.icustay_id = 
mv.icustay_id where 
"""
features = ['ss.age', 'ss.is_male', 'ss.race_white', 'ss.race_black', 'ss.race_hispanic','ss.race_hispanic', 
            'ss.race_other', 'ss.height', 'mv.weight']

for i in features:
    query_sepsis +=  i + ' is not null and '
query_sepsis = query_sepsis[:-len(' and ')]

    
query_pivoted_lab = '''
select * from public.pivoted_lab where icustay_id is not null
'''

sql_statement = '''
select * from (
with ss as (
'''\
+ query_sepsis + ' ),  pvl as (' \
+ query_pivoted_lab + ') '\
+'''
select pvl.subject_id as subject_id, ss.icustay_id as icustay_id, ss.hadm_id as hadm_id,
ss.age as age, ss.is_male as is_male, ss.race_white as race_white, ss.race_black as race_black, ss.race_hispanic 
as race_hispanic, ss.race_other as race_other, ss.height as height, ss.weight as weight, 
pvl.charttime as pvl_charttime, pvl.aniongap as aniongap, pvl.albumin as albumin, pvl.bands as bands,
pvl.bicarbonate as bicarbonate, pvl.bilirubin as bilirubin, pvl.creatinine as creatinine,pvl.chloride as chloride,
pvl.hematocrit as hematocrit,pvl.hemoglobin as hemoglobin,pvl.lactate as lactate,
pvl.platelet as platelet,pvl.potassium as potassium,pvl.ptt as ptt,pvl.inr as inr,pvl.pt as pt,
pvl.sodium as sodium,pvl.bun as bun,pvl.wbc as wbc 
from ss join  pvl on ss.hadm_id=pvl.hadm_id
) cohort where cohort.icustay_id in (select distinct icustay_id from public.chartevents where itemid = 683 or itemid = 224684)

''' 



ppt_df = pd.read_sql_query(sql_statement,conn) 
ppt_df

,subject_id,icustay_id,hadm_id,age,is_male,race_white,race_black,race_hispanic,race_other,height,...,hemoglobin,lactate,platelet,potassium,ptt,inr,pt,sodium,bun,wbc
0,3,211552,145834,76.5246,1,1,0,0,0,179.07,...,10.0,NaN,282.0,5.4,30.7,1.3,13.5,136.0,53.0,12.7
1,3,211552,145834,76.5246,1,1,0,0,0,179.07,...,NaN,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,211552,145834,76.5246,1,1,0,0,0,179.07,...,NaN,8.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,211552,145834,76.5246,1,1,0,0,0,179.07,...,7.8,NaN,190.0,4.0,58.3,1.7,15.7,143.0,41.0,11.3
4,3,211552,145834,76.5246,1,1,0,0,0,179.07,...,8.2,NaN,253.0,4.1,46.4,1.7,15.6,140.0,43.0,19.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270572,99995,229633,137810,88.6974,0,1,0,0,0,159.00,...,NaN,NaN,NaN,NaN,34.7,1.6,17.1,NaN,NaN,NaN
270573,99995,229633,137810,88.6974,0,1,0,0,0,159.00,...,NaN,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270574,99995,229633,137810,88.6974,0,1,0,0,0,159.00,...,NaN,NaN,NaN,3.8,NaN,NaN,NaN,138.0,12.0,NaN
270575,99995,229633,137810,88.6974,0,1,0,0,0,159.00,...,8.9,NaN,124.0,4.2,33.3,1.6,17.4,137.0,11.0,6.4


In [4]:
def median_impute(df, columns):
    # df has column icustay_id
    for col in columns:
        median = df[col].dropna(how='any').median() # global median
        stay_median = {}
        for stay,val in zip(df['icustay_id'], df[col]):
            if stay not in stay_median:
                stay_median[stay] = []
            if not math.isnan(float(val)):
                stay_median[stay].append(float(val))
        stay_median1 = {}
        for i in stay_median:
            vals = stay_median[i]
            if len(vals) == 0:
                stay_median1[i] = median # global median
            else:
                stay_median1[i] = np.median(vals) # local median
        new_values = []
        for stay,val in zip(df['icustay_id'], df[col]):
            if math.isnan(float(val)):
                new_values.append(stay_median1[stay])
            else:
                new_values.append(float(val))
        df[col] = new_values
    return df

def datetime_to_sec(df,time_col='charttime'):
    time_in_sec = []
    date_format = '%Y-%m-%d %H:%M:%S'
    for i in df[time_col]:
        time_in_sec.append(datetime.strptime(str(i),date_format).timestamp())
    df[time_col] = time_in_sec
    return df

def datetime_to_sec_entry(time):
    date_format = '%Y-%m-%d %H:%M:%S'
    return datetime.strptime(str(time),date_format).timestamp()

In [5]:
ppt_df = datetime_to_sec(ppt_df, time_col='pvl_charttime')
columns = ['aniongap', 'albumin','bands','bicarbonate','bilirubin','creatinine','chloride', 'hematocrit',\
          'hemoglobin', 'lactate', 'platelet', 'potassium', 'ptt', 'inr', 'pt', 'sodium', 'bun', 'wbc'] # drop glucose
ppt_df = median_impute(ppt_df, columns)
ppt_df

,subject_id,icustay_id,hadm_id,age,is_male,race_white,race_black,race_hispanic,race_other,height,...,hemoglobin,lactate,platelet,potassium,ptt,inr,pt,sodium,bun,wbc
0,3,211552,145834,76.5246,1,1,0,0,0,179.07,...,10.0,2.6,282.0,5.4,30.7,1.3,13.50,136.0,53.0,12.7
1,3,211552,145834,76.5246,1,1,0,0,0,179.07,...,9.7,4.3,191.0,4.1,74.1,1.5,14.65,137.5,32.0,12.5
2,3,211552,145834,76.5246,1,1,0,0,0,179.07,...,9.7,8.8,191.0,4.1,74.1,1.5,14.65,137.5,32.0,12.5
3,3,211552,145834,76.5246,1,1,0,0,0,179.07,...,7.8,2.6,190.0,4.0,58.3,1.7,15.70,143.0,41.0,11.3
4,3,211552,145834,76.5246,1,1,0,0,0,179.07,...,8.2,2.6,253.0,4.1,46.4,1.7,15.60,140.0,43.0,19.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270572,99995,229633,137810,88.6974,0,1,0,0,0,159.00,...,8.9,1.1,166.0,3.9,34.7,1.6,17.10,137.5,11.5,7.3
270573,99995,229633,137810,88.6974,0,1,0,0,0,159.00,...,8.9,0.9,166.0,3.9,34.0,1.6,17.40,137.5,11.5,7.3
270574,99995,229633,137810,88.6974,0,1,0,0,0,159.00,...,8.9,1.1,166.0,3.8,34.0,1.6,17.40,138.0,12.0,7.3
270575,99995,229633,137810,88.6974,0,1,0,0,0,159.00,...,8.9,1.1,124.0,4.2,33.3,1.6,17.40,137.0,11.0,6.4


### Extract vital signs and impute

In [6]:
def feature_dict(df, feature_start=2, stay=0, time=1):
    result = {}
    n = len(df)
    for i in range(n):
        row = list(df.iloc[i])
        stay_id = row[stay]
        charttime = row[time]
        features = row[feature_start:]
        if stay_id not in result:
            result[stay_id] = {}
        result[stay_id][charttime] = features
    return result 

file = './vital_df.pickle'
if os.path.exists(file):
    file = open(file,'rb')
    vital_dict = pickle.load(file) # {'icustay_id:{charttime1:[features], charttime2:[features]}'}
else:    
    stay_id = []
    for i in ppt_df['icustay_id']:
        stay_id.append(int(i))


    query_vital = 'select * from public.pivoted_vital where icustay_id in (select icustay_id from public.sepsis3)'
    vital_df = pd.read_sql_query(query_vital,conn) 
    exclude = []
    row = 0
    for i in vital_df['icustay_id']:
        if int(i) not in stay_id:
            exclude.append(row)
        row += 1
    vital_df.drop(exclude) # [icustay_id, charttime, 'heartrate', 'sysbp','diasbp','meanbp','resprate','tempc','spo2','glucose']
    columns = ['heartrate', 'sysbp','diasbp','meanbp','resprate','tempc','spo2','glucose']
    vital_df = median_impute(vital_df,columns)
    vital_df = datetime_to_sec(vital_df)
    vital_dict = vital_df = feature_dict(vital_df, feature_start=2) # {'icustay_id:{charttime1:[features], charttime2:[features]}'}
    file = open(file,"wb")
    pickle.dump(vital_dict, file)

### Convert lab DF into dictionary

In [7]:
stay_start = {}
for i,j in zip(ppt_df['icustay_id'], ppt_df['pvl_charttime']):
    if i not in stay_start:
        stay_start[i] = []
    stay_start[i].append(j)
for i in stay_start:
    stay_start[i] = min(stay_start[i])

columns = ['subject_id','icustay_id', 'hadm_id', 'pvl_charttime','age', 'is_male', 'race_white', 'race_black', \
           'race_hispanic', 'race_other',  'height','weight', \
           'aniongap', 'albumin','bands','bicarbonate','bilirubin','creatinine',\
           'chloride', 'hematocrit','hemoglobin', 'lactate', 'platelet', 'potassium', 'ptt', 'inr', 'pt', \
           'sodium', 'bun', 'wbc']
ppt_dict = ppt_df = feature_dict(ppt_df[columns], feature_start=4, stay=1, time=3) # [subject_id, icustay_id, hadm_id,...]
# {'icustay_id:{charttime1:[features], charttime2:[features]}'}

## Merge lab and vital signs

In [8]:
structural_features= {} # {'icustay_id:{charttime1:[features], charttime2:[features]}'}
for stay in stay_start:
    start = stay_start[stay]
    lab_times = sorted(ppt_dict[stay].keys())
    temp = {}
    for vital_time in vital_dict[stay]:
        idx = bisect.bisect_left(lab_times, vital_time) - 1
        if idx < 0:
            continue
        lab_time = lab_times[idx]
        merged_features = copy.deepcopy(ppt_dict[stay][lab_time])
        merged_features.extend(vital_dict[stay][vital_time])
        temp[vital_time] = merged_features
    if len(temp) == 0:
        continue
    else:
        structural_features[stay] = temp

#del ppt_dict
#del vital_dict

## Extract treatment

In [9]:
file = './treatment.pickle'
if os.path.exists(file): #os.path.exists(file)
    file = open(file,'rb')
    treatment_dict = pickle.load(file) #  {icustay_id:{time1:dosage1, time2:dosage2}}
else:
    treatment_query = '''
    select distinct ce.icustay_id, ce.charttime as starttime, (ce.valuenum / w.weight) as dosage_ml_kg 
    from public.chartevents ce join public.weightfirstday w on ce.icustay_id = w.icustay_id 
    where (ce.itemid=683 or ce.itemid = 224684) and ce.valuenum is not null and w.weight is not null
    and w.icustay_id in (select distinct icustay_id from public.ventilation_classification where mechvent = 1)
    '''
    treatment_df = pd.read_sql_query(treatment_query,conn)
    treatment_dict = {}
    n = len(treatment_df)
    for i in range(n):
        icustay_id, time, amount = list(treatment_df[['icustay_id', 'starttime', 'dosage_ml_kg']].iloc[i])
        if icustay_id not in stay_start:
            continue
        else:
            if icustay_id not in treatment_dict:
                treatment_dict[icustay_id] = {}
            treatment_dict[icustay_id][datetime_to_sec_entry(time)] = amount
    del treatment_df
    file = open(file,"wb")
    pickle.dump(treatment_dict, file)

## Extract outcome

In [10]:
file = './outocmes.pickle'
if os.path.exists(file): #os.path.exists(file)
    file = open(file,'rb')
    outcome_dict = pickle.load(file) #  {icustay_id:{time1:SAS1, time2:SAS2}}
else:
    outcome_query = ''' 

select * from (
with co as
(select icu.subject_id, icu.icustay_id, vent.starttime - interval '1' day as starttime,
 vent.starttime + interval '2' hour as endtime from
 public.icustays icu join public.ventilationdurations vent on icu.icustay_id = vent.icustay_id
)

,
s2 as (

  select co.ICUSTAY_ID, ce.CHARTTIME
    -- pre-process the FiO2s to ensure they are between 21-100%
    , max(
        case
          when itemid = 223835
            then case
              when valuenum > 0 and valuenum <= 1
                then valuenum * 100
              -- improperly input data - looks like O2 flow in litres
              when valuenum > 1 and valuenum < 21
                then null
              when valuenum >= 21 and valuenum <= 100
                then valuenum
              else null end -- unphysiological
        when itemid in (3420, 3422)
        -- all these values are well formatted
            then valuenum
        when itemid = 190 and valuenum > 0.20 and valuenum < 1
        -- well formatted but not in %
            then valuenum * 100
      else null end
    ) as fio2_chartevents
  from co
  left join public.CHARTEVENTS ce
    on co.icustay_id = ce.icustay_id
    and ce.ITEMID in
    (
      3420 -- FiO2
    , 190 -- FiO2 set
    , 223835 -- Inspired O2 Fraction (FiO2)
    , 3422 -- FiO2 [measured]
    )
    group by co.ICUSTAY_ID, ce.CHARTTIME

)

,bg as 
(
    select pvt.ICUSTAY_ID, pvt.CHARTTIME
  , max(case when label = 'SPECIMEN' then value else null end) as SPECIMEN
  , max(case when label = 'FIO2' then valuenum else null end) as FIO2
  , max(case when label = 'PO2' then valuenum else null end) as PO2
  from
  ( -- begin query that extracts the data
    select co.icustay_id, charttime
    -- here we assign labels to ITEMIDs
    -- this also fuses together multiple ITEMIDs containing the same data
        , case
          when itemid = 50800 then 'SPECIMEN'
          when itemid = 50816 then 'FIO2'
          when itemid = 50821 then 'PO2'
          else null
          end as label
          , value
          -- add in some sanity checks on the values
          , case
              when valuenum <= 0 then null
              when itemid = 50816 and valuenum > 100 then null -- FiO2
              -- conservative upper limit
              when itemid = 50821 and valuenum > 800 then null -- PO2
          else valuenum
          end as valuenum

      from co
      left join public.labevents le
        on co.subject_id = le.subject_id
        and le.charttime between co.starttime and co.endtime
        and le.ITEMID in (50800, 50816, 50821)
  ) pvt
  group by pvt.icustay_id, pvt.CHARTTIME
  -- we only want rows with a PO2 measurement
  having max(case when label = 'PO2' then valuenum else null end) is not null
)
select
  bg.icustay_id, bg.charttime
  , bg.PO2, bg.FIO2, s2.fio2_chartevents
  , case when coalesce(bg.FIO2, s2.fio2_chartevents, 0) > 0 and coalesce(bg.FIO2, s2.fio2_chartevents, 100) < 100
        then 100*bg.PO2/(coalesce(bg.FIO2, s2.fio2_chartevents))
      else null end as pao2fio2
  , ROW_NUMBER() over (partition by bg.icustay_id order by bg.charttime DESC, s2.charttime DESC) as rn
from bg
left join s2
  -- same patient
  on  bg.icustay_id = s2.icustay_id
  -- fio2 occurred at most 4 hours before this blood gas
  and s2.charttime between bg.charttime - interval '4' hour and bg.charttime
where coalesce(bg.SPECIMEN,'ART') = 'ART'
) pfr where pao2fio2 is not null
'''
    outcome_df = pd.read_sql_query(outcome_query,conn)
    outcome_dict = {}
    n = len(outcome_df)
    for i in range(n):
        icustay_id, time, ss = list(outcome_df[['icustay_id', 'charttime', 'pao2fio2']].iloc[i])
        if icustay_id not in stay_start:
            continue
        else:
            if icustay_id not in outcome_dict:
                outcome_dict[icustay_id] = {}
            outcome_dict[icustay_id][datetime_to_sec_entry(time)] = ss
    file = open(file,"wb")
    pickle.dump(outcome_dict, file)
    del outcome_df
    

## Merge features, dosage, outcome

In [23]:
## structural_features= {}  {'icustay_id:{charttime1:[features], charttime2:[features]}'}
## treatment_dict = {} {icustay_id:{time1:dosage1, time2:dosage2}}
## outcome_dict = {} {icustay_id:{time1:SAS1, time2:SAS2}}
## stay_start {icustay_id: starttime}
## inclusion criteria: sepsis cohort, patient with mechanical ventilation. Use dosage as 
## tidal volume (set) (mL/kg). PF ration should be recorded within some time after treatment.

unique_stay = set()
all_data = []
maxtime_after_treatment = 20*60 # sec, the maximum time after treatment that the label should be recorded
recorded_label = {} # {icustay_id:[label time1, label time 2,...]} use each label only once
icu_rowid = []

for stay in treatment_dict:
    if stay not in structural_features or stay not in outcome_dict:
        continue        
        
    for treat_start in treatment_dict[stay]:
        temp = []
        # find features before treatment
        feature_times = sorted(list(structural_features[stay]))
        idx = bisect.bisect_left(feature_times, treat_start) - 1
        if idx < 0:
            continue
        features = structural_features[stay][feature_times[idx]]
        # find dosage
        dosage = treatment_dict[stay][treat_start]
        
        label_times = sorted(list(outcome_dict[stay]))
        idx = bisect.bisect_left(label_times, treat_start) - 1
        if idx < 0:  ###+++
            continue ###+++
        label_before_t = outcome_dict[stay][label_times[idx]] # label before treatment ###+++
        
        try:
            label_time = label_times[idx + 1]
        except:
            continue # no label
        # check if label is recorded too late
        if label_time - treat_start > maxtime_after_treatment:
            continue
        if stay not in recorded_label:
            recorded_label[stay] = []
        else:
            if label_time in recorded_label[stay]:
                continue
        recorded_label[stay].append(label_time)
        label_after_t = outcome_dict[stay][label_time]
        temp.append(label_after_t)
        temp.append(dosage)
        temp.extend(features)
        temp.append(label_before_t) ###+++
        all_data.append(temp)
        unique_stay.add(stay)
        icu_rowid.append(stay)

### Store data

In [29]:
for i in [structural_features, treatment_dict, outcome_dict, stay_start]:
    del i
def write_csv(outpath, data):
    with open(outpath, 'w',newline='') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerows(data)
file = './mimiciii_mv_raw.csv'
write_csv(file, all_data)

### Find unique patient

In [15]:
sepsis_query = '''
select icu.subject_id, ss.icustay_id from public.sepsis3 ss join public.icustays icu on 
ss.icustay_id=icu.icustay_id
'''
sepsis_df = pd.read_sql_query(sepsis_query,conn)
sepsis_dict = {} # {icustay_id:subject_id}
n = len(sepsis_df)
for i in range(n):
    subject, stay = list(sepsis_df[['subject_id', 'icustay_id']].iloc[i])
    if stay not in sepsis_dict:
        sepsis_dict[stay] = []
    sepsis_dict[stay].append(subject)
    
unique_subject = set()
for i in unique_stay:
    for j in sepsis_dict[i]:
        unique_subject.add(j)
print(('The number of unique subject: ', len(unique_subject)))

('The number of unique subject: ', 309)


## Extract Hypertension

In [26]:
query = '''
select distinct icu.icustay_id from public.icustays icu join public.diagnoses_icd icd on 
icu.hadm_id = icd.hadm_id where icd.icd9_code like '401%'

'''
hyper_df = pd.read_sql_query(query,conn)
selected_icuid = []
hyper = list(hyper_df['icustay_id'])
for idx in icu_rowid:
    if idx in hyper:
        selected_icuid.append(1)
    else:
        selected_icuid.append(0)
path = './hypertension.pickle'
file = open(path,"wb")
pickle.dump(selected_icuid, file)


In [30]:
sum(selected_icuid)

114